## Vektory slov

V našem předchozím příkladu jsme pracovali s vysoko-dimenzionálními vektory bag-of-words o délce `vocab_size` a explicitně jsme převáděli nízko-dimenzionální poziční reprezentace na řídké jednorozměrné reprezentace. Tato jednorozměrná reprezentace není paměťově efektivní. Navíc je každé slovo považováno za nezávislé na ostatních, takže jednorozměrné kódování nevyjadřuje sémantické podobnosti mezi slovy.

V této části budeme pokračovat v průzkumu datasetu **News AG**. Nejprve načteme data a získáme některé definice z předchozí části.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Co je to embedding?

Myšlenka **embeddingu** spočívá v reprezentaci slov pomocí nízkodimenzionálních hustých vektorů, které odrážejí sémantický význam slova. Později si povíme, jak vytvořit smysluplné word embeddings, ale prozatím si embeddingy představme jako způsob, jak snížit dimenzionalitu vektorů slov.

Embedding vrstva tedy přijímá slovo jako vstup a produkuje výstupní vektor o specifikované velikosti `embedding_size`. V jistém smyslu je velmi podobná vrstvě `Dense`, ale místo toho, aby přijímala jednorozměrný vektor jako vstup, dokáže přijmout číslo odpovídající slovu.

Použitím embedding vrstvy jako první vrstvy v naší síti můžeme přejít od modelu bag-of-words k modelu **embedding bag**, kde nejprve převedeme každé slovo v našem textu na odpovídající embedding a poté vypočítáme nějakou agregační funkci nad všemi těmito embeddingy, například `sum`, `average` nebo `max`.

![Obrázek ukazující embedding klasifikátor pro pět slov v sekvenci.](../../../../../translated_images/cs/embedding-classifier-example.b77f021a7ee67eee.webp)

Naše neuronová síť klasifikátoru se skládá z následujících vrstev:

* Vrstva `TextVectorization`, která přijímá řetězec jako vstup a produkuje tenzor čísel tokenů. Určíme rozumnou velikost slovníku `vocab_size` a ignorujeme méně často používaná slova. Vstupní tvar bude 1 a výstupní tvar bude $n$, protože získáme $n$ tokenů jako výsledek, přičemž každý z nich obsahuje čísla od 0 do `vocab_size`.
* Vrstva `Embedding`, která přijímá $n$ čísel a redukuje každé číslo na hustý vektor o dané délce (v našem příkladu 100). Tím se vstupní tenzor tvaru $n$ transformuje na tenzor $n\times 100$.
* Agregační vrstva, která vypočítá průměr tohoto tenzoru podél první osy, tj. vypočítá průměr všech $n$ vstupních tenzorů odpovídajících různým slovům. K implementaci této vrstvy použijeme vrstvu `Lambda` a předáme jí funkci pro výpočet průměru. Výstup bude mít tvar 100 a bude představovat číselnou reprezentaci celé vstupní sekvence.
* Konečný lineární klasifikátor `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


V přehledu, ve sloupci **tvar výstupu**, první rozměr tenzoru `None` odpovídá velikosti minibatch, zatímco druhý odpovídá délce sekvence tokenů. Všechny sekvence tokenů v minibatch mají různé délky. O tom, jak s tím pracovat, si povíme v další části.

Teď pojďme natrénovat síť:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Poznámka** že vytváříme vektorizér na základě podmnožiny dat. To se provádí za účelem urychlení procesu, což může vést k situaci, kdy ne všechny tokeny z našeho textu budou přítomny ve slovníku. V takovém případě budou tyto tokeny ignorovány, což může vést k mírně nižší přesnosti. Nicméně, v reálném životě podmnožina textu často poskytuje dobrý odhad slovníku.


### Práce s různými délkami sekvencí proměnných

Pojďme si vysvětlit, jak probíhá trénink v minibatchích. V uvedeném příkladu má vstupní tensor rozměr 1 a používáme minibatche o délce 128, takže skutečná velikost tensoru je $128 \times 1$. Počet tokenů v každé větě je však odlišný. Pokud aplikujeme vrstvu `TextVectorization` na jeden vstup, počet vrácených tokenů se liší v závislosti na způsobu tokenizace textu:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Nicméně, když aplikujeme vektorizér na několik sekvencí, musí vytvořit tenzor obdélníkového tvaru, takže nevyužité prvky vyplní tokenem PAD (který je v našem případě nula):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Zde můžeme vidět vnoření:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Poznámka**: Aby se minimalizovalo množství doplňování, v některých případech má smysl seřadit všechny sekvence v datové sadě podle rostoucí délky (nebo přesněji podle počtu tokenů). To zajistí, že každý minibatch bude obsahovat sekvence podobné délky.


## Sémantické vektorizace: Word2Vec

V našem předchozím příkladu se vrstva pro vektorizaci naučila mapovat slova na jejich vektorové reprezentace, avšak tyto reprezentace neměly sémantický význam. Bylo by užitečné naučit se vektorovou reprezentaci, kde podobná slova nebo synonyma odpovídají vektorům, které jsou si blízké podle určité vzdálenosti mezi vektory (například euklidovské vzdálenosti).

Abychom toho dosáhli, musíme náš model pro vektorizaci předtrénovat na velké kolekci textů pomocí techniky, jako je [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Tato metoda je založena na dvou hlavních architekturách, které se používají k vytvoření distribuované reprezentace slov:

 - **Continuous bag-of-words** (CBoW), kde trénujeme model tak, aby předpovídal slovo na základě okolního kontextu. Pokud máme n-gram $(W_{-2},W_{-1},W_0,W_1,W_2)$, cílem modelu je předpovědět $W_0$ na základě $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** je opakem CBoW. Model využívá okolní okno kontextových slov k předpovědi aktuálního slova.

CBoW je rychlejší, zatímco skip-gram je pomalejší, ale lépe reprezentuje méně častá slova.

![Obrázek ukazující algoritmy CBoW a Skip-Gram pro převod slov na vektory.](../../../../../translated_images/cs/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Pro experimentování s Word2Vec vektorizací předtrénovanou na datasetu Google News můžeme použít knihovnu **gensim**. Níže najdeme slova nejpodobnější slovu 'neural'.

> **Note:** Když poprvé vytváříte slovní vektory, jejich stahování může chvíli trvat!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Můžeme také extrahovat vektorové zakotvení ze slova, které bude použito při trénování klasifikačního modelu. Zakotvení má 300 komponent, ale zde ukazujeme pouze prvních 20 komponent vektoru pro přehlednost:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Skvělá věc na sémantických vnořeních je, že můžete manipulovat s vektorovým kódováním na základě sémantiky. Například můžeme požádat o nalezení slova, jehož vektorová reprezentace je co nejblíže slovům *král* a *žena*, a co nejdále od slova *muž*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Příklad výše používá určité interní kouzlo GenSym, ale základní logika je ve skutečnosti docela jednoduchá. Zajímavou věcí na vnořeních je, že můžete provádět běžné operace s vektory na vektorových reprezentacích vnoření, což odráží operace na **významech** slov. Příklad výše lze vyjádřit pomocí vektorových operací: vypočítáme vektor odpovídající **KRÁL-MUŽ+ŽENA** (operace `+` a `-` jsou prováděny na vektorových reprezentacích odpovídajících slov) a poté najdeme nejbližší slovo ve slovníku k tomuto vektoru:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Museli jsme přidat malé koeficienty k vektorům *man* a *woman* – zkuste je odstranit a podívejte se, co se stane.

K nalezení nejbližšího vektoru používáme TensorFlow mechanismus k výpočtu vektoru vzdáleností mezi naším vektorem a všemi vektory ve slovníku, a poté najdeme index minimálního slova pomocí `argmin`.


Zatímco Word2Vec se zdá být skvělým způsobem, jak vyjádřit sémantiku slov, má mnoho nevýhod, včetně následujících:

* Modely CBoW i skip-gram jsou **prediktivní vektorizace**, které berou v úvahu pouze lokální kontext. Word2Vec nevyužívá globální kontext.
* Word2Vec nezohledňuje **morfologii** slov, tj. skutečnost, že význam slova může záviset na různých částech slova, jako je kořen.

**FastText** se snaží překonat druhé omezení a staví na Word2Vec tím, že se učí vektorové reprezentace pro každé slovo a n-gramy znaků obsažené v každém slově. Hodnoty těchto reprezentací jsou pak při každém kroku trénování zprůměrovány do jednoho vektoru. I když to přidává spoustu dodatečných výpočtů během předtrénování, umožňuje to vektorům slov zakódovat informace o podslovech.

Další metoda, **GloVe**, používá odlišný přístup k vektorizaci slov, založený na faktorizaci matice slovního kontextu. Nejprve vytvoří velkou matici, která počítá počet výskytů slov v různých kontextech, a poté se snaží tuto matici reprezentovat v nižších dimenzích tak, aby minimalizovala ztrátu při rekonstrukci.

Knihovna gensim podporuje tyto vektorizace slov a můžete s nimi experimentovat změnou kódu pro načítání modelu výše.


## Použití předtrénovaných embeddingů v Keras

Můžeme upravit výše uvedený příklad tak, aby byla matice v naší embedding vrstvě předem naplněna sémantickými embeddingy, jako je Word2Vec. Slovníky předtrénovaného embeddingu a textového korpusu se pravděpodobně nebudou shodovat, takže si musíme vybrat jeden. Zde prozkoumáme dvě možné možnosti: použití slovníku tokenizeru a použití slovníku z embeddingů Word2Vec.

### Použití slovníku tokenizeru

Při použití slovníku tokenizeru budou některá slova ze slovníku mít odpovídající embeddingy Word2Vec, zatímco jiná budou chybět. Vzhledem k tomu, že velikost našeho slovníku je `vocab_size` a délka vektorů embeddingu Word2Vec je `embed_size`, bude embedding vrstva reprezentována váhovou maticí tvaru `vocab_size`$\times$`embed_size`. Tuto matici naplníme tak, že projdeme slovník:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Pro slova, která nejsou přítomna ve slovníku Word2Vec, můžeme buď ponechat jejich hodnoty jako nuly, nebo vygenerovat náhodný vektor.

Nyní můžeme definovat vrstvu pro vkládání s předem naučenými váhami:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Poznámka**: Všimněte si, že jsme nastavili `trainable=False` při vytváření `Embedding`, což znamená, že vrstvu Embedding nebudeme znovu trénovat. To může způsobit mírně nižší přesnost, ale urychluje to proces trénování.

### Použití slovníku pro embedding

Jedním z problémů předchozího přístupu je, že slovníky použité v TextVectorization a Embedding jsou odlišné. Abychom tento problém překonali, můžeme použít jednu z následujících možností:
* Znovu natrénovat Word2Vec model na našem slovníku.
* Načíst náš dataset se slovníkem z předtrénovaného Word2Vec modelu. Slovníky použité k načtení datasetu lze specifikovat během načítání.

Druhý přístup se zdá být jednodušší, takže ho implementujeme. Nejprve vytvoříme vrstvu `TextVectorization` se specifikovaným slovníkem, který je převzat z Word2Vec embeddingů:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Knihovna word embeddings Gensim obsahuje praktickou funkci `get_keras_embeddings`, která automaticky vytvoří odpovídající vrstvu embeddings pro Keras.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Jedním z důvodů, proč nevidíme vyšší přesnost, je to, že některá slova z našeho datového souboru chybí v předtrénovaném slovníku GloVe, a proto jsou v podstatě ignorována. Abychom to překonali, můžeme na základě našeho datového souboru natrénovat vlastní vektory slov.


## Kontextuální vektory slov

Jedním z hlavních omezení tradičních předtrénovaných reprezentací vektorů slov, jako je Word2Vec, je skutečnost, že i když dokážou zachytit určitý význam slova, nedokážou rozlišit mezi různými významy. To může způsobovat problémy v následných modelech.

Například slovo 'play' má odlišný význam v těchto dvou větách:
- Šel jsem na **hru** do divadla.
- John si chce **hrát** se svými přáteli.

Předtrénované vektory, o kterých jsme mluvili, reprezentují oba významy slova 'play' stejným způsobem. Abychom toto omezení překonali, musíme vytvářet vektory založené na **jazykovém modelu**, který je natrénován na velkém korpusu textu a *rozumí*, jak mohou být slova spojována v různých kontextech. Diskuze o kontextuálních vektorech slov přesahuje rámec tohoto tutoriálu, ale vrátíme se k nim při probírání jazykových modelů v další části.



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
